In [1]:
import torch
import string
import random
import re
import time, math

import torch
import torch.nn as nn
from torch.autograd import Variable

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [2]:
# Ler um arquivo em texto puro contendo o conteúdo a ser aprendido
file = open('lfv_texts_plain.txt').read()
file_len = len(file)
print('file_len =', file_len)

all_characters = ''.join(set(file))
n_characters = len(all_characters)

print(all_characters, n_characters)

file_len = 1298370
―p̀êakyì"4wî-?'’ocífHAFÉ*7%ḩó–VJõKàÇ…(ŕNWbR̈‘8$ãU“uYéáç1X5E!,0;S.ei
â2°@ À nCÕt9gjú̂sdmTD—)`3x”6:ÁG~üLvÕÓlPZqèÍôÃzMI&/QB 123


In [3]:
chunk_len = 200

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

foi a desforra de Pelé, um lance da sua biografia que ele gentilmente compartilhou com o Brasil.
Na Copa de 74 o Brasil ainda vivia sob um regime militar, mas tínhamos uma forte razão sentimental pa


In [5]:
# Converte string para uma lista de inteiros

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
      try:
        tensor[c] = all_characters.index(string[c])
      except:
        print(c)
        raise
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([ 4, 43, 17, 91, 60, 22])


In [6]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [ ]:
def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)
    

def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

In [7]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [8]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = rnn.init_hidden().cuda()
    prime_input = char_tensor(prime_str).cuda()
    predicted = prime_str

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [31]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden.cuda()
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [10]:
n_epochs = 50000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].cuda(),example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Os', 100), '\n')


 23s (100 0%) 2.2114]
Os curo come sesa ic camem eipelia onco comam dora que saba desti
pra em a ancundo á dó ho-em com que 

 46s (200 0%) 2.0441]
Os uma achar pele com aperde teram e tora deconumo elá ela vime racê. Vocuma ́us o velita vais, que  

 9s (300 0%) 1.9525]
Os, porma lora? Astas. O no do estuma da perso trinhentes as nos fastado se para do a pócia ampelidá 

 32s (400 0%) 2.3421]
Os sea fense alguma eu e cinha tem para endetava amous vezente entem para parere, era para falar, estu 

 57s (500 1%) 1.9210]
Os das de vel infuca as ao deises ― prirente não entre para de como hamente. Conhe de amaram a sua ca 

 20s (600 1%) 2.0806]
Os como orioso.
Síverado de for que o toino é um a dia homeros e verão, contricar o ficada? O dite  

 45s (700 1%) 1.7680]
Os de como os pauleira do Maras não eslivers, envez do sentio os dizer de uma madir que faito robia.  

 11s (800 1%) 1.8271]
Os Wim 12 mas infonhas mas em que o sera um aridas.
– Preguldo na pelo antéria, mesoro pa

 55s (6400 12%) 1.4852]
Os depois, pelo nos nos certeza, é que eu querem escadidos sobre o poder só entre as produziram mais 

 20s (6500 13%) 1.4745]
Os já estava talvez sem atendiram até se represa o determiro para a natura.
― Brase disse a tudo…
— Q 

 43s (6600 13%) 1.5397]
Os que os portamento do Paris. Já estavam seria na vida. Eu deixa as coisas vendos e que Arientes que 

 5s (6700 13%) 1.2912]
Os transmitir não dá uma fora do seu casco do Zexo da maneira em porta vez, mas na polesta da Branqu 

 28s (6800 13%) 1.3484]
Os
Nada por inventou a mena mesmo acompanhando apareceu a proporta é dirada. Eu não andar a Filha da 

 50s (6900 13%) 1.5325]
Os inventores contramos na ponto de seis se maridos deve mais os computadas de comum do pótrico, os o 

 14s (7000 14%) 1.5259]
Os confordem, crianços e precisavam nada procômica do chego, as monogras. Não diria ser o que está 

 38s (7100 14%) 1.2644]
Os não havia piu com a próprises afinal, mas que ele pinta ― próloga num

 53s (12600 25%) 1.0790]
Os nossas setas no farítico nos seus motoristas em que as convencidos para perderia serem só de algu 

 16s (12700 25%) 1.2077]
Os ameaços do homemon canaláuia ter uma coisa dizer que eu não podia se fora mais constantária, qu 

 38s (12800 25%) 1.2729]
Os representos da frase. Somos é o que pouco mimens de lembrar uma brilha Malufney ― Sua aquela razã 

 1s (12900 25%) 1.5056]
Os diassem, tentava muito baixa na cidade. A esqueção (ão tinha administração daqui que se dar um 

 24s (13000 26%) 1.3405]
Os vícios para o homem é ele era nós, a Julinha. Ele também poderia relevanta, a loja das abanas e 

 47s (13100 26%) 1.6649]
Os meus? Isso televisão, mas ele consegue pular, não se encontrou de paixão teria, saiu do cachorro 

 11s (13200 26%) 1.3515]
Os jabos e sem palavra presentei um aceito com retrebol estar com o seu desolver o outro, o pôquer di 

 34s (13300 26%) 1.3158]
Os nomes é o nome da ocasião e não pensou na Beatriz, numa mulh

 23s (18800 37%) 1.2842]
Os churrascos nos altaria e o Além do teclado? Aliás, com coração no médico e com seus meses depo 

 45s (18900 37%) 1.3960]
Os contrários e não tiver na sua chada. Uma baile? Não tem antídoto.
Pois depois de larajá-la com 

 8s (19000 38%) 1.4519]
Os de sempre uma branco Isfercina. Mas acontece. Algo ouvi-me a cerveja para o chapéu, a empresa e se 

 31s (19100 38%) 1.4810]
Os bríncos.
― Qual ficou na conversa. ― O quê?
― Não, não é o que nos anos de áfasa. Nunca teria 

 54s (19200 38%) 1.2254]
Os anos de 8000 anos de interguntas tantos suicido em que a botar a mulher e de um caninal de esperanz 

 16s (19300 38%) 1.4823]
Os descascos de proporções com ele. Gente era um truque. Um surpres começou a contatou todos naturais 

 39s (19400 38%) 1.3469]
Os dois não se permírios começou a amada com tudo que como ele permance que o assato, eu nunca faze 

 1s (19500 39%) 1.1277]
Os dois se estágicas cantadas nuas tem mata para uma rainha do alt

 16s (25000 50%) 1.2583]
Os defensões de sim; Espar ganha, começando as crianças e os outros é que o valor de contra os ver 

 39s (25100 50%) 1.3494]
Os burços. Eu sei senhor em que o fim do Diretor que se respirar do mundo e preciso de uma resposta na 

 2s (25200 50%) 1.2850]
Os Estavam tudo que se aprenderam as mesmas artes da Orgem da teoria não pendura da terra precisar te 

 25s (25300 50%) 1.3458]
Os anjos sobre os outros poucos foradas, de prédios já estavam sóbrios de direções em que também 

 48s (25400 50%) 1.3833]
Os pernas com o estado acho que a cidade de pé do lar e não derror do mesmo tempo passageiro das ter 

 11s (25500 51%) 1.3838]
Os novos sempre e os dois primitivos pelos outros de exageros, também tivessem posições da grandes  

 34s (25600 51%) 1.2369]
Os comoços entre os comunistas e as cartas para a seita. Só queria ter seguinte, como o próprio jog 

 57s (25700 51%) 1.0767]
Os que não têm continuado pelo quando o primeiro nome de mão em

 54s (31100 62%) 1.2054]
Os álibis e nenhum dos mais solitários sobre o pai e sabendo seu seu encontrador com uma cabeleira d 

 17s (31200 62%) 1.1276]
Os dedos, lampas a como o mundo, declarou que o caratura da patricidade de verão e ela sabe com eles  

 40s (31300 62%) 1.2652]
Os dias de pedros e uma pecassada da calçada.
Na vida ficou chamado Jose. Espere um Barbosa. E comece 

 3s (31400 62%) 1.2656]
Os príncipes remotos são lutas e mais vestigam até apenas dos gatos estava acabados. Muito bigodent 

 26s (31500 63%) 1.2372]
Os relaças. Tanto tempo para um HQEH. Um gato e prova com a cascuda de rei, inconsciente contou a ord 

 49s (31600 63%) 1.3199]
Os banheiros constituídos eram decidir o pouco?
― Se devolve ter que o valência? Era muito colete. E 

 12s (31700 63%) 1.1438]
Os dedos era a apoteose!
― Estou invadiu?
― Então ― diz o marquês. ― Arrã! Não, ou cabeça com int 

 34s (31800 63%) 1.1910]
Os dedos. Ele que a Maria Alice tive ou não tenhora acontecido pe

 46s (37200 74%) 1.1952]
Os nomes, em coisa que já tem um desconhecido. Ele quer chegar tem um casal, e seus milégios são el 

 9s (37300 74%) 1.2477]
Os gatos semos pensar num conto. Use as cochicadas suficientes. Só nada para merece de falá-lo com c 

 31s (37400 74%) 0.8173]
Os mesmos?
Ele para pronunca provar livrinhado de um mosco e ao bar do homem se alarmar para vista com 

 54s (37500 75%) 1.3197]
Os antigos que não existem os foltas filme. A própria nossa festa com uma defesa, a sua inútelipeza 

 16s (37600 75%) 1.2408]
Os cientistas e a frente depois da Copa e um pai da boca os anos esquimes são mais em alegoria, e a c 

 38s (37700 75%) 1.4813]
Os jovens vivem de outro livre. Havia ter alguns, por escapagem por alimentar o conordem e a respeitou 

 1s (37800 75%) 1.2869]
Os barcos secretas se apaixonados. Não será concessão de uma revéias. No caso, como provável. Com 

 23s (37900 75%) 1.0290]
Os dois podemos ter sido centro de Pamela, o que na pedra do seu ce

 25s (43300 86%) 1.3323]
Os dois são impossíveis de Robert Allex. Se a idéia de levar o almoço. Os minutos eram para o rest 

 46s (43400 86%) 1.2162]
Os outros suspirando
Cláudia para ver com o que os dois catigos para acertecer. Ela disse "Meu corpo  

 8s (43500 87%) 1.2030]
Os fogos para você tinha uma varidade ou seu carmano são olho para o que faz mais provescionar numa co 

 29s (43600 87%) 1.3483]
Os condições para proteger a sete de uma operação do grande identido do céu, e todos estavam desa 

 51s (43700 87%) 1.3827]
Os homens ficam combinados interiores indeficientes para a cadeia da hora a vida, por nada, pelo dia d 

 13s (43800 87%) 1.1105]
Os dois ajustiam, como não faziam todos estão confiantes. Confessivo dele e uma canivinha?
Finalment 

 35s (43900 87%) 1.1466]
Os casos chegamos a espaço, quando pediram fácil. Todo o mundo era do Guingura e internacional, sua  

 57s (44000 88%) 1.1412]
Os dois provavelmente bolas: ele acontecera. Só se entrava a sua m

 43s (49400 98%) 1.3164]
Os homens com a returcenar era destruirando de carne em todos dedos. As duas costas assim daquele jogo 

 5s (49500 99%) 1.3145]
Os Pafusos e amelando bom seres começando de emergêques apenas sempre a ser os longe. Vamos defender 

 26s (49600 99%) 1.0266]
Os decades raramente passando a sua mulhida o mundo. Tenho uma enterrada mais dúvida de Trabalhos fur 

 48s (49700 99%) 1.3182]
Os sapatos que dará muito jogador, casa e é essa volta do Vinícius, que tinha postício do mundo, o 

 11s (49800 99%) 1.2522]
Os condições do outro constante encargente e estava contando em no literatura que a Senhora Cristã  

 33s (49900 99%) 1.4610]
Os vezes conseguimos apesar de destruir que os outros ficam os discos de Todos os daqueles dias dizend 

 55s (50000 100%) 1.2238]
Os homens e o claro ou observador na Anda bica desembarcar da História, no ar, aos dondozeiros anos.  



In [11]:
# Com a rede treinada, gerar um texto longo

print(evaluate('O psicanalista de', 5500), '\n')

O psicanalista deles estava! Está bem com saber o meu prestence da escrivência e completo o homem. Dois milhores do primeiro objetivo disse aquele alma que a sua vida, nem confiar nos lugares reculosos, da armação em silêncio. ― Que garrafasíssimo. ― Que nós contem. O velho é um cartãozinho. Se não se recusou um movimento. Mas você fosse roubalhado. Só não tinha se melhor moderna. ― É. Está me seguido que o Raul não tenho cena da seleção e permasse. ― Como é vestir que o José, ou por acaso não têm malditos outras esposes. Vai ser receita. Mas eu não me ouve qualquer tempo e agora num selvagente! Estão na sua morte. E se antes de ser hemenda.
― E no meu pai. ― Não, não. Mudar melhor que daria todas. Nada. Só por quê?
― Ô, pedindo a Mario e Vitor Hugo. Pelo menos não é de discussar a toda. E contente. Não estava dizer em que me visto o guarana. ― É com todo o mundo!
― Você não se instamos o problemas...
― Que queixar de ter o que vai falar com a Juna?
―

# Agora utilizando LSTM

In [33]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.lstm(input.view(1, 1, -1), hidden)
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden

    def init_hidden(self):
        return (Variable(torch.zeros(self.n_layers, 1, self.hidden_size)), 
                Variable(torch.zeros(self.n_layers, 1, self.hidden_size)))

In [34]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = rnn.init_hidden()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    prime_input = char_tensor(prime_str).cuda()
    predicted = prime_str

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [35]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [36]:
n_epochs = 50000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].cuda(), example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Os', 100), '\n')


 29s (100 0%) 2.7290]
Os" dro alhem pos, pom Mre e Loum ea sa comida queŕ rem,rita uivtovanta m ceta os E peseri  o ve,, do 

 57s (200 0%) 2.3396]
Oso mase des que os meMm ?u erita sontão tão diu le Vhara caliita ma Gonteta de que som ara qué ten 

 26s (300 0%) 2.1425]
Os se ererialdia. Vode cocuridára emos vrezeremo ilucoitis, lmamenmersa estere feme, cere alsas.
― A  

 56s (400 0%) 2.1932]
Os grestanos naHar., Não o, a de mocesão é am os Ler-a vara no inteceros paí̃o nos felão um daiis 

 27s (500 1%) 2.0692]
Os socinha ganhá.
― Quranhera de cadicez e etê sentios sais de, de palandos. "Enemprá.
―.
― Om pora 

 58s (600 1%) 2.0723]
Os 1ridigregaial. Vecilia a cataro. ― ― ― O cazecive?
― Astam o ninomer Pois ê pora discouda diltarou 

 28s (700 1%) 1.9836]
Os improsta são da esima ao viada de que de segulado, nentro é bala esma devas não e ó é trama qu 

 57s (800 1%) 2.3478]
Os girodo, sermósedo.
2́ o se nfinha Tisse se que era a tizido, que a mésse mos de teri

 8s (6400 12%) 1.6525]
Os lempos de Brigê nom-se apresentemente se tinha reação de que a Sica é se certo de Maria Tatiza  

 38s (6500 13%) 1.6604]
Os entre uma respantas aliversam eles de que ser continua do khando sabe da yragua não essa, antes co 

 7s (6600 13%) 1.5694]
Os movimento de amigo com os teriados para telebreifante. Para alimentaram chegaram da compreira com e 

 38s (6700 13%) 1.5783]
Os partidos da final. Exaga de mesma filha de gariam ― indesiline. Quem ter abraça é liga o filoso sa 

 8s (6800 13%) 1.6209]
Os exemplos e sentir para interessos da mãe da conhece de pérua seqüessera a histeros o me direito. 

 40s (6900 13%) 1.5171]
Os anos, pefso", e a às que eram sua voz ineracional na cheiro. Não teria um família de fachorrar a  

 10s (7000 14%) 1.4263]
Os filhos, na baixa de pretendiram a mesma linguar de para aqui estava ver o que digidez que menei voc 

 40s (7100 14%) 1.5751]
Os senhoras os incomunhos entroníocas sem se perguntou não me agorava não

 48s (12600 25%) 1.2657]
Os entendidos aduminar seu seu bom internelo. Amores e você perdia de nós prevovocavam. Anos com ela. 

 17s (12700 25%) 1.3160]
Os chonias. E também pudim pela roua. ― E dava adeixa. ― Se, em companheiro. Sua descrever que nunca  

 45s (12800 25%) 1.3530]
Os destruderas de anjos. Depois à crime nos cuça dos entresse: ― Deve trandôbia-se que a recentualid 

 13s (12900 25%) 1.4371]
Os infernos ficou a vida do que tinha sair de comista, não saíram com as crises de inglês, sabendo  

 42s (13000 26%) 1.2561]
Os encontraram com uma conversa na pessoas de verdade para um poucos. Que entendia na caminha na roupa 

 11s (13100 26%) 1.7476]
Os esperanças de sarrabamente, mas bem depois e para a mitade das suas sincerá como um graços e os  

 39s (13200 26%) 1.3903]
Os Marios ― mais passeanos que faz nascer sua trabalho e já conseguir uma casa se alguma pretender a  

 8s (13300 26%) 1.4096]
Os principalmente com suminhas por fazer o vílio em coisa, para s

 17s (18800 37%) 1.1905]
Os amigos e lembra e não tem jantar como músicos e fazeram o meilofos era tendo à noite de tanto e  

 46s (18900 37%) 1.4900]
Os outros virtos brilhos, mandada, um gesto era solto, no ser encontro do general.
E o melhor depois,  

 15s (19000 38%) 1.2102]
Os sandrás que pode ser uma redência. Vou de escrever na decridão de um lado de "Esperando" e eu di 

 43s (19100 38%) 1.2811]
Os gritos. Depois de Pércico, ela seja preguei contra o tio uma méida. Não ser chamado a parte de a 

 13s (19200 38%) 1.2844]
Os anos. Mas no primeiro tempo de manter estranheza. Não quilega, pense se vê em contrabilidade. Seu 

 41s (19300 38%) 1.3170]
Os nádores, cujo aquilo.
– Como você não é que o "Dia Palf) Nada. Mal. A cabeça. Ele estava por p 

 9s (19400 38%) 1.1948]
Os nomes que vinhos viratas de teve na calção hoje, mas a gargar, perfeito.
Ela estava existe vizinh 

 38s (19500 39%) 1.2377]
Os jantares ou um bom causo, meu filhos, sentada para contá-las ―

 25s (24900 49%) 1.2197]
Os paraísos, continua é que a Mariá contundo, que tencara ser dizer que por missar o tipo. Que, tir 

 54s (25000 50%) 1.3190]
Os que eles não foram uma parte e ciruína com ele. Mas vem apertamento numa coisa que nem Roudy Bein 

 24s (25100 50%) 1.1921]
Os dois garotos e morrer e sentido, mamãe: conselhará como a exploseira. Mas Maria Matilde era tarde 

 54s (25200 50%) 1.6071]
Os nossos serem calarnas pra ouvir buscar o carnaval sujeito. Tenho bem. Não quer estranho de tanto p 

 22s (25300 50%) 1.0743]
Os universos e eu não consegue gente que ele não lhes era se as passagemos. Notádia do Proi era pel 

 51s (25400 50%) 1.3605]
Os decitas aos outros amores e ele estava livres, Deus morreram, e que até planejei a privade da Alin 

 20s (25500 51%) 1.3150]
Os.
― Pois é...
― Pois atrás de petreso entre a corte de misturismos e não haveria acorda.
― Você  

 49s (25600 51%) 1.2961]
Os sorrisos e brasileiros, no paresser a sua rotição. O convoço

 44s (31000 62%) 1.2063]
Os conquistadores e estava visto.
— Quero saber por qualquer coisa!
— Certo.
— Corpo conta que tinha u 

 13s (31100 62%) 1.1469]
Os velhos segundos, a dormir. A arte chamarem a um cachorro ou do primeiro e onde foi se então a prov 

 41s (31200 62%) 1.0625]
Os termos quando a mulher para ele sair a cabeça com eles. Tinha sido o outro, e na praia com um pala 

 11s (31300 62%) 1.2634]
Os alvos expressos de algum toco de descobrir que eu. Onde estava mais perfeita de Termes. O trabalhar 

 39s (31400 62%) 1.1585]
Os escrevers e das continuosos foi dizer "Eu vou centro e nem poder de todos os seus pai, começariam  

 8s (31500 63%) 1.1978]
Os autor e se aproveituais em "Boete para nobre" com trás da tia há perplexão do que fazer e me ret 

 37s (31600 63%) 1.3306]
Os papagaios.
Antes do Cicatriz estava além chato de Mesta de garante do braço.
― A testa épologia  

 6s (31700 63%) 1.1595]
Os. A Artur estão ainda chorado para Dilitan. Como alguma pervocaç

 18s (37100 74%) 1.7979]
Os que encontrava no seu celular. O Endereço, a assassinativa e lá na porta no resto. Desde uma mulhe 

 48s (37200 74%) 1.3318]
Os azulenas então no mesmo peito, com celular a goiana, a porta.
Faisandê ― Você conheceram que o B 

 19s (37300 74%) 1.2879]
Os semanas e nos receberam, o proximador da Calicute e Justibilidade. Todas as concordando num copo fi 

 50s (37400 74%) 1.1981]
Os dominam
para certamente, existe, livram o Murilinho está de que o memório de desconhecer que o di 

 21s (37500 75%) 1.2769]
Os existindo ditas. É o triste fora. Sinatra ou seu corteiro dos filhos e ficaria os dois também ser 

 51s (37600 75%) 1.1540]
Os homens para assistir as duas propostas de Paraíso.
Como faria isso dificuldade e o assessor que um 

 22s (37700 75%) 1.2233]
Os conhecimentos, ou o seu serviço perfeita e para ser o espaço da máscara intimigada do seu assao  

 52s (37800 75%) 1.4332]
Os equivaletros hilitos do primeiro movimento só de exama de esc

 49s (43200 86%) 1.0997]
Os conheços dos meus anos.
― Não sei não, não é sair do caso ― disse Colon. Tinha unsinado com o  

 18s (43300 86%) 1.1556]
Os outros suspeitas do meu casamento com Marinho com qualquer grande cigarro. Depois sendo que o senho 

 46s (43400 86%) 1.2339]
Os moedas inconseqüentes, e trabalhando no caso de mortamentos ou um pouco começaria a defesa, uma c 

 15s (43500 87%) 1.1929]
Os carajetas da praia e o casal de nos filhos gregados em ponticular sua parde de dona.
O leitor solid 

 44s (43600 87%) 1.1088]
Os universos do mundo e Loberdar para com ele, em que casarão as carcandadas. E quando cabem grandade 

 15s (43700 87%) 1.1656]
Os esse seus bolsos internacionais, não em salvar em com uma das suas firmas no fim da maneira. A tel 

 43s (43800 87%) 1.1388]
Os livros pretos que se repete a capacidade francesa, com a banda de precisar de mais experimas da ten 

 11s (43900 87%) 1.2774]
Os filhos do cachorro.
O meio de um compreendente preferir olhar 

 56s (49300 98%) 1.1836]
Os botões, a sua mulher do cidadão na hora do Hitler perguntou seu apartamento, na Ita, meu amor no  

 25s (49400 98%) 1.3883]
Os capitos de cortar o que pelo apartamento poderia estar no interpretação de um churrasco do Lineu, 

 54s (49500 99%) 1.1129]
Os tempos extremos famosos.
O prefeito da mentira que nos passaram para os pacíficos. Sizem que nariz 

 23s (49600 99%) 1.0045]
Os, um abanano com outros olhos. Você ainda dava botar uma conversa como podia estar em que a gente f 

 52s (49700 99%) 1.2083]
Os três poucos anos, para entregar as radicais, pula onde todo para a luz e a Branquinha, roubou de f 

 21s (49800 99%) 0.9553]
Os casamentos ao superáculo que a vizinha do Picapática Mação inconsolável, naquela sala. Depois  

 50s (49900 99%) 1.3590]
Os apelos despendesse mais alto sobre a encontrar o que fez muito no meu eto de baixo para ser o nosso 

 19s (50000 100%) 1.0748]
Os écios se continuamos dirigente que mais fará natural não t

In [38]:
# Com a rede treinada, gerar um texto longo

print(evaluate('O psicanalista de', 5500), '\n')

O psicanalista de se levará todo mesmo que tinha sido doís. Os que eles não tinham ao surdo melhor de areias e suas juras para a escolha com uma respectiva do grupo. Constituinte do comértimo e desta com o brasileiro Eisente, só pode ser a beijada por completo e deixou com que adestirar o caro e serviu a leve que teste pedisse que ela era que o vestido desta grande tirada. Eu não estou você na esquina do Gastão fora a continuaria a todo o fato e, que podia mais do confundimo com a Romáca. E ela não estava na pista que disse:
― Primeira coisa do que ele está ciúme porque sim. O que me enganava que ele a minha vida have um mês? ― crem- do que o Fu atiro.
― Preto...
― É morava. De espada, com a história. Vem um assédio pelo Lilian no chão, lima a casa com ela e meia me vê o meu queixo do meu pai. Eu não pode ser um jeito com o Fernando que era... Uma arte e eu sou. Ela estava de descobrir o que servem. Vamos no Brasil. Eu não estava com seu comportário do seu período